In [68]:
import aiohttp
import asyncio
import bs4
import datetime
from io import BytesIO
import pandas as pd
import PyPDF2
import requests
import time
from tqdm.contrib.telegram import tqdm
from transliterate import to_latin
import json
import os

In [69]:
TOKEN = os.getenv('TOKEN')
CHAT_ID = os.getenv('CHAT_ID')

In [70]:
world_url = "https://uzreport.news/world?page={page}" #1250
world_last = 880

politics_url = "https://uzreport.news/politics?page={page}"
politics_last = 670

society_url = "https://uzreport.news/society?page={page}"
society_last = 2690

economy_url = "https://uzreport.news/economy?page={page}"
economy_last = 1010

finance_url = "https://uzreport.news/finance?page={page}"
finance_last = 575

insurance_url = "https://uzreport.news/insurance?page={page}"
insurance_last = 150

exchenge_url = "https://uzreport.news/exchange?page={page}"
exchenge_last = 160

technology_url = "https://uzreport.news/technology?page={page}"
technology_last = 180

culture_url = "https://uzreport.news/culture?page={page}"
culture_last = 330

sports_url = "https://uzreport.news/sports?page={page}"
sports_last = 1060

football_url = "https://uzreport.news/football?page={page}"
football_last = 535

analytics_url = "https://uzreport.news/analytics?page={page}"
analytics_last = 3

In [71]:
URLS = [[world_url, world_last], [politics_url, politics_last], [society_url, society_last], [analytics_url, analytics_last], 
        [economy_url, economy_last], [finance_url, finance_last], [insurance_url, insurance_last], [exchenge_url, exchenge_last],
        [technology_url, technology_last], [culture_url, culture_last], [sports_url, sports_last], [football_url, football_last]]

In [72]:
BASE_URL = "https://uzreport.news"

In [73]:
links = []

In [54]:
async with aiohttp.ClientSession() as session:
    async with session.get("https://uzreport.news/lang/oz") as resp:
        headers = resp.headers
        print(headers)

<CIMultiDictProxy('Server': 'nginx', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Cache-Control': 'no-cache, private', 'Date': 'Fri, 21 Jul 2023 12:28:37 GMT', 'Set-Cookie': 'XSRF-TOKEN=eyJpdiI6IkdVMUxPdTRaRkIwblpRTzhCNWdkMlE9PSIsInZhbHVlIjoiQXYxUmRZa2pwRExkQUU0VmJ6MzJTYVY3Vk9vbTdiWDNRUUVFdERhS3ByOUNQTGFoNisxcUluOVVqY3o2bDlQcnFvTzJkNlpXMWZOSTR0S3dlK3kwV1E9PSIsIm1hYyI6Ijk5YjZkZDc3YjA4M2Q2MDEyYjliMTg1ZDMyYTlkZGM1ZWQ4NDVjZjI3MjdmZjY1ZDQ3MzQ0M2U1ZGMzNTQ2M2EifQ%3D%3D; expires=Fri, 21-Jul-2023 21:08:37 GMT; Max-Age=31200; path=/', 'Set-Cookie': 'uzreport=eyJpdiI6InF1eWdyYUo3b29jYVQxeFpwUWYwZEE9PSIsInZhbHVlIjoiUDRyaDRMeTVadmpDNFd3bmp4bHNkZmwzVmY5T3pwd1FxUUU4ZUtWc0lBVVAxMjNURjBqbmJrZ1Jzdk42R0tZYnZEbUJGS1AxaUJ2bG1GVVZlUDVEXC93PT0iLCJtYWMiOiI1ZjZkNTY3ZGJmMTE1NDk5ZTQ4MmE2MzgyNTI3M2FmMWNkMjVhYzJmYTk5NzlmYjFmZTE5ZjlkOWNjZGZlMmI4In0%3D; expires=Fri, 21-Jul-2023 21:08:37 GMT; Max-Age=31200; path=/; httponly', 'Content-Encoding': 'gzip')>


In [64]:
cookie = []

for header in headers.items():
    if header[0] == 'Set-Cookie':
        cookie.append(header[1].split(";")[0])

In [66]:
cookie_text = "; ".join(cookie)
cookie_text

'XSRF-TOKEN=eyJpdiI6IkdVMUxPdTRaRkIwblpRTzhCNWdkMlE9PSIsInZhbHVlIjoiQXYxUmRZa2pwRExkQUU0VmJ6MzJTYVY3Vk9vbTdiWDNRUUVFdERhS3ByOUNQTGFoNisxcUluOVVqY3o2bDlQcnFvTzJkNlpXMWZOSTR0S3dlK3kwV1E9PSIsIm1hYyI6Ijk5YjZkZDc3YjA4M2Q2MDEyYjliMTg1ZDMyYTlkZGM1ZWQ4NDVjZjI3MjdmZjY1ZDQ3MzQ0M2U1ZGMzNTQ2M2EifQ%3D%3D; uzreport=eyJpdiI6InF1eWdyYUo3b29jYVQxeFpwUWYwZEE9PSIsInZhbHVlIjoiUDRyaDRMeTVadmpDNFd3bmp4bHNkZmwzVmY5T3pwd1FxUUU4ZUtWc0lBVVAxMjNURjBqbmJrZ1Jzdk42R0tZYnZEbUJGS1AxaUJ2bG1GVVZlUDVEXC93PT0iLCJtYWMiOiI1ZjZkNTY3ZGJmMTE1NDk5ZTQ4MmE2MzgyNTI3M2FmMWNkMjVhYzJmYTk5NzlmYjFmZTE5ZjlkOWNjZGZlMmI4In0%3D'

In [75]:
links = []

In [ ]:
async def main():
    headers = {
              'Content-Type': 'application/json',
              'Cookie': 'XSRF-TOKEN=eyJpdiI6IkdVMUxPdTRaRkIwblpRTzhCNWdkMlE9PSIsInZhbHVlIjoiQXYxUmRZa2pwRExkQUU0VmJ6MzJTYVY3Vk9vbTdiWDNRUUVFdERhS3ByOUNQTGFoNisxcUluOVVqY3o2bDlQcnFvTzJkNlpXMWZOSTR0S3dlK3kwV1E9PSIsIm1hYyI6Ijk5YjZkZDc3YjA4M2Q2MDEyYjliMTg1ZDMyYTlkZGM1ZWQ4NDVjZjI3MjdmZjY1ZDQ3MzQ0M2U1ZGMzNTQ2M2EifQ%3D%3D; uzreport=eyJpdiI6InF1eWdyYUo3b29jYVQxeFpwUWYwZEE9PSIsInZhbHVlIjoiUDRyaDRMeTVadmpDNFd3bmp4bHNkZmwzVmY5T3pwd1FxUUU4ZUtWc0lBVVAxMjNURjBqbmJrZ1Jzdk42R0tZYnZEbUJGS1AxaUJ2bG1GVVZlUDVEXC93PT0iLCJtYWMiOiI1ZjZkNTY3ZGJmMTE1NDk5ZTQ4MmE2MzgyNTI3M2FmMWNkMjVhYzJmYTk5NzlmYjFmZTE5ZjlkOWNjZGZlMmI4In0%3D'
            }
    
    
    async with aiohttp.ClientSession() as session:
        
        for url in URLS:
            URL = url[0]
            LAST = url[1]
            
            for page in tqdm(range(1, LAST+1), token=TOKEN, chat_id=CHAT_ID):
                try:
                    
#                     async with session.get(URL.format(page=page), headers = headers) as resp:
#                         if resp.status == 200:
#                             data = await resp.text()
#                             soup = bs4.BeautifulSoup(data, "lxml")
                            
                    
                    async with session.get(URL.format(page=page), headers = headers) as resp:
                        if resp.status == 200:
                            data = await resp.text()
                            soup = bs4.BeautifulSoup(data, "lxml")
                            
                            lang = soup.find("html")['lang']
                            if lang != "oz":
                                async with session.get("https://uzreport.news/lang/oz") as resp:
                                    headers = resp.headers
                                    
                                    cookie = []
                                    for header in headers.items():
                                        if header[0] == 'Set-Cookie':
                                            cookie.append(header[1].split(";")[0])
                                    cookie_text = "; ".join(cookie)
                                    
                                    headers = {
                                              'Content-Type': 'application/json',
                                              'Cookie': cookie_text
                                                }
                                    
                                    continue
                            
                            selected_links = soup.find('div', class_='search-content hidden-xs').find_all("div", class_="col-xs-12")
                            links.extend([link.find("a")['href'] for link in selected_links])
                            
                        elif resp.status != 404:
                            await asyncio.sleep(60) # 1 minute
                except Exception as e:
                    print(e, "page_num", page)
await main()

  0%|          | 0/880 [00:00<?, ?it/s]

  0%|          | 0/670 [00:00<?, ?it/s]

In [ ]:
with open(r'./HelperFiles/uzreportUz.txt', 'w') as fp:
    for item in links:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

In [8]:
len(links)

7904

In [15]:
links

['https://uzreport.news/world/lg-planiruet-zakrit-svoy-zavod-v-rossii-i-perenesti-ego-v-druguyu-stranu',
 'https://uzreport.news/world/yujnokoreyskiy-hyundai-motor-hochet-prodat-svoy-zavod-v-rossii',
 'https://uzreport.news/world/voz-predstavila-platformu-elektronniy-vrach-kotoriy-daet-meditsinskie-konsultatsii-na-7-ya',
 'https://uzreport.news/world/moshniy-vzriv-i-obval-proletov-podrobnosti-podriva-gruzovogo-avtomobilya-na-krimskom-mostu',
 'https://uzreport.news/world/evrovidenie-2023-sostoitsya-v-britanskom-liverpule',
 'https://uzreport.news/world/ochnaya-vstrecha-putina-i-zelenskogo-mojet-sostoyatsya-v-noyabre',
 'https://uzreport.news/world/na-ostrove-pashi-iz-za-pojara-postradali-znamenitie-kamennie-idoli',
 'https://uzreport.news/world/azerbaydjan-i-armeniya-do-kontsa-etogo-goda-mogut-podpisat-mirniy-dogovor',
 'https://uzreport.news/world/nobelevskiy-komitet-obyavil-laureatov-premii-mira-2022',
 'https://uzreport.news/world/kim-chen-yin-pozdravil-vladimira-putina-s-70-letiem'

In [67]:
# requests.get("https://uzreport.news/lang/oz")

headers = {
  'Content-Type': 'application/json',
  'Cookie': 'XSRF-TOKEN=eyJpdiI6IkdVMUxPdTRaRkIwblpRTzhCNWdkMlE9PSIsInZhbHVlIjoiQXYxUmRZa2pwRExkQUU0VmJ6MzJTYVY3Vk9vbTdiWDNRUUVFdERhS3ByOUNQTGFoNisxcUluOVVqY3o2bDlQcnFvTzJkNlpXMWZOSTR0S3dlK3kwV1E9PSIsIm1hYyI6Ijk5YjZkZDc3YjA4M2Q2MDEyYjliMTg1ZDMyYTlkZGM1ZWQ4NDVjZjI3MjdmZjY1ZDQ3MzQ0M2U1ZGMzNTQ2M2EifQ%3D%3D; uzreport=eyJpdiI6InF1eWdyYUo3b29jYVQxeFpwUWYwZEE9PSIsInZhbHVlIjoiUDRyaDRMeTVadmpDNFd3bmp4bHNkZmwzVmY5T3pwd1FxUUU4ZUtWc0lBVVAxMjNURjBqbmJrZ1Jzdk42R0tZYnZEbUJGS1AxaUJ2bG1GVVZlUDVEXC93PT0iLCJtYWMiOiI1ZjZkNTY3ZGJmMTE1NDk5ZTQ4MmE2MzgyNTI3M2FmMWNkMjVhYzJmYTk5NzlmYjFmZTE5ZjlkOWNjZGZlMmI4In0%3D'
}

resp = requests.get(world_url.format(page=6), headers=headers)

data = resp.text
soup = bs4.BeautifulSoup(data)

lang = soup.find('html')['lang']
selected_links = soup.find('div', class_='search-content hidden-xs').find_all("div", class_="col-xs-12")
print(selected_links)
print(lang)

# content = soup.find('div', class_="center_panel").find_all('p')

# print(content)

[<div class="col-xs-12">
<div class="col-lgright40">
<div class="heading_newsbox long">
<div class="row flex">
<div class="col-xs-5 col-sm-6 col-md-6 col-lg-3">
<a href="https://uzreport.news/world/janubiy-koreya-prezidenti-ukrainaga-bordi">
<img alt="Janubiy Koreya prezidenti Ukrainaga bordi" src="https://uzreport.news/fotobank/small/image/12a0ed5edc250a077da2fa449ed43ab1.jpeg" title="Janubiy Koreya prezidenti Ukrainaga bordi"/>
</a>
</div>
<div class="col-xs-7 col-sm-6 col-md-6 col-lg-9 heading_newsbox_txt">
<h3><a href="https://uzreport.news/world/janubiy-koreya-prezidenti-ukrainaga-bordi">Janubiy Koreya prezidenti Ukrainaga bordi</a></h3>
<p></p>
<div class="heading_newsbox_txt_nav">
<div>
</div>
<div>
<ul>
<li class="time">
<a>
<i aria-hidden="true" class="fa fa-clock-o"></i>
										12:51, 15 iyul 2023 yil									</a>
</li>
<li class="rubric"><a href="https://uzreport.news/world">Dunyoda</a></li>
<li class="icons">
<div class="comments"><a href="#news-comment"><i class="fa fa-

In [ ]:
results = []

async def main():
    async with aiohttp.ClientSession() as session:  
        
        for ind, link in enumerate(tqdm(links, token=TOKEN, chat_id=CHAT_ID)):
            try:
                async with session.get(link) as resp:
                    if resp.status == 200:
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)

                        content = soup.find('div', class_="center_panel").find_all('p')
                        title = soup.find('div', class_="center_panel").find("h1").text

                        content = "".join(i.text for i in content[1:])
                        results.append({
                            "title": title.strip(),
                            "content": content.strip()
                        })
                    elif resp.status != 404:
#                         await asyncio.sleep(60) # 1 minute
                        print("404", link)
            except Exception as e:
#                 print(title)
#                 print(content)
                print(e, link)
    
await main()

In [ ]:
df = pd.DataFrame(results)

In [25]:
from transliterate import to_latin

df['title'] = df['title'].apply(lambda x: to_latin(x))

df['content'] = df['content'].apply(lambda x: to_latin(x))

In [ ]:
df.to_pickle("MainFiles/uzreportUz.pickle")

In [14]:
df

,title,content
0,LG планирует закрыть свой завод в России и пер...,Южнокорейский производитель бытовой техники LG...
1,Южнокорейский «Hyundai Motor» хочет продать св...,Южнокорейский производитель автомобилей Hyunda...
2,"ВОЗ представила платформу «электронный врач», ...",Всемирная организация здравоохранения представ...
3,Мощный взрыв и обвал пролетов: подробности под...,Сегодня утром на автомобильной части Крымского...
4,«Евровидение-2023» состоится в британском Ливе...,Международный песенный конкурс «Евровидение-20...
...,...,...
60674,В Ташкенте отметили день футбола и дружбы,
60675,Миржалол Касымов назначен главным тренером ФК ...,Генеральный директор футбольного клуба “Бунёдк...
60676,Встречайте телеканал FUTBOL TV!,"Как известно, учитывая пожелания многомиллионн..."
60677,"ЛЧА: ""Локомотив"" и ""Истиклол"" сыграли вничью –...",11 апреля в Ташкенте состоялась футбольный мат...


In [81]:
with open('HelperFiles/uzreport.txt', 'r') as file:
    ru_links = [book.replace("\n", "") for book in file.readlines()]

In [82]:
with open('HelperFiles/uzreportUz.txt', 'r') as file:
    uz_links = [book.replace("\n", "") for book in file.readlines()]

In [84]:
for link in ru_links:
    if link in uz_links:
        print(link)